In [ ]:
import climpy.aeronet_utils as aeronet
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import datetime as dt

__author__ = 'Sergey Osipov <Serega.Osipov@gmail.com>'

# get list of all Aeronet stations
file_paths = aeronet.get_stations_file_path(aeronet.ALL_STATIONS, level=15, res=aeronet.ALL_POINTS, inv=False)


def spatial_filter(file_path):
    """
    Example of the spatial filter.
    Checks if the station is inside the polygon.
    :param file_path:
    :return:
    """
    df = aeronet.read_aeronet(file_path, only_head=True)
    point = Point(df['Site_Longitude(Degrees)'].iloc[0], df['Site_Latitude(Degrees)'].iloc[0])
    # MENA region
    polygon = Polygon([(30, 12), (30, 30), (60, 30), (60, 12)])
    return polygon.contains(point)


def spatial_and_temporal_filter(file_path):
    """
    Example of the spatial and temporal filter.
    Checks if the station is inside the polygon and if time coverage is appropriate.
    :param file_path:
    :return:
    """
    df = aeronet.read_aeronet(file_path, only_head=True)
    point = Point(df['Site_Longitude(Degrees)'].iloc[0], df['Site_Latitude(Degrees)'].iloc[0])
    # MENA region
    polygon = Polygon([(30, 12), (30, 30), (60, 30), (60, 12)])
    loc_flag = polygon.contains(point)
    time_flag = df.index[0] <= dt.datetime(2019, 1, 1)
    # TODO: we can also read last record and compare the right time interval as well

    return loc_flag and time_flag


# get list of stations inside the polygon
fps, dfs = aeronet.filter_stations(file_paths, spatial_filter)

# or add additional time filter
fps, dfs = aeronet.filter_stations(file_paths, spatial_and_temporal_filter())

In [ ]:
# now you can loop through them
# for fp in fps:
#     print(fp)

#The previous results depend on what you have downloaded
# TODO: Alternative approach is too scrape the web page with the list of stations and coordinates
# for example here https://aeronet.gsfc.nasa.gov/cgi-bin/draw_map_display_aod_v3
# but the page results (filters) are not URL specific


# given each station
fp = fps[0]
# you can read all fields into pandas
df = aeronet.read_aeronet(fp)

# or instead get station name
station = fp.split('/')[-1]
# and use aux function to return only AOD as ValueObject
aeronet_aod_vo = aeronet.get_aod_diag(station, 'AOD_500nm', level=15, res=aeronet.ALL_POINTS)
# you can also apply time filter
time_range = (dt.datetime(2016, 1, 1), dt.datetime(2016, 2, 1))
aeronet_aod_vo = aeronet.get_aod_diag(station, 'AOD_500nm', level=15, res=aeronet.ALL_POINTS,
                                      time_range=time_range)


